In [18]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,VotingRegressor
from sklearn.model_selection import KFold,cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler,PolynomialFeatures,RobustScaler
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.pipeline import Pipeline,make_pipeline
from xgboost import XGBRegressor
import lightgbm as lgb
import warnings
import numpy as np
df_full = pd.read_csv('prepared.csv')

X_train = df_full.iloc[:1453].drop(columns='SalePrice')
y_train = df_full.iloc[:1453].SalePrice
X_test = df_full.iloc[1453:].drop(columns='SalePrice')
X_train.drop(['Id'], axis=1, inplace=True)
X_test.drop(['Id'], axis=1, inplace=True)
ID = df_full.iloc[1453:].Id
df_full.head(1)

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley_Grvl,Alley_None,Alley_Pave,BedroomAbvGr,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,...,YrSold_2006,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010,has2ndfloor,hasbsmt,hasfireplace,hasgarage,haspool
0,5.939034,1025.651979,0.0,0,1,0,3,1,0,0,...,0,0,1,0,0,1,1,0,1,0


In [5]:
n_folds = 5
def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train.values)
    rmse= np.sqrt(-cross_val_score(model, X_train.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [29]:
# alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
# ridge = make_pipeline(RobustScaler(),RidgeCV(alphas=alphas_alt, cv=kfolds))
# result = rmsle_cv(ridge)
# result.mean()
# print('result'.format(result.mean()))

result


In [ ]:
lasso = Pipeline([('poly', PolynomialFeatures()), 
                  ('model', Lasso(max_iter=1e7,))])

ridge = Ridge(alpha=13)
elastic = ElasticNet()
predicitons = rmsle_cv(lasso)
print(predicitons.mean())
predicitons = rmsle_cv(elastic)
print(predicitons.mean())

In [47]:
clf = VotingRegressor([('Lassp', lasso),('Ridge',ridge)])
predictions = rmsle_cv(clf)
predictions.mean()

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.021958224601086, tolerance: 0.018758569046793183
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.721436515385034, tolerance: 0.01763895249214213
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.592698889208142, tolerance: 0.018034543670964215
  positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.85

0.10376099845198469

12.01347834343976

In [71]:
df = pd.DataFrame( {'ID': ID, 'SalePrice': np.expm1(predictions)})
df.set_index("ID",inplace=True)
df.to_csv('submission.csv')